<a href="https://colab.research.google.com/github/navidh86/perturbseq-10701/blob/master/baseline_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # ONLY FOR COLAB
# !git clone https://github.com/navidh86/perturbseq-10701.git
# %cd ./perturbseq-10701
# !pip install fastparquet tqdm

In [1]:
# Imports and device
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
import numpy as np

from data.reference_data_classification import get_dataloader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


In [2]:
# # Create dataloaders (point to data/ paths explicitly)
# train_loader = get_dataloader(
#     parquet_path='data/tf_gene_expression_labeled.parquet',
#     tf_sequences_path='data/tf_sequences.pkl',
#     gene_sequences_path='data/gene_sequences_4000bp.pkl',
#     batch_size=128,
#     type='train',
#     majority_fraction=0.01
# )
# test_loader = get_dataloader(
#     parquet_path='data/tf_gene_expression_labeled.parquet',
#     tf_sequences_path='data/tf_sequences.pkl',
#     gene_sequences_path='data/gene_sequences_4000bp.pkl',
#     batch_size=256,
#     type='test',
#     majority_fraction=0.01
# )

# print('Train size:', len(train_loader.dataset))
# print('Test size :', len(test_loader.dataset))

# Create dataloaders (point to data/ paths explicitly)
train_loader = get_dataloader(
    parquet_path='data/tf_gene_expression_labeled_v2.parquet',
    tf_sequences_path='data/tf_sequences.pkl',
    gene_sequences_path='data/gene_sequences_4000bp.pkl',
    batch_size=128,
    type='train',
    majority_fraction=0.005
)

validation_loader = get_dataloader(
    parquet_path='data/tf_gene_expression_labeled_v2.parquet',
    tf_sequences_path='data/tf_sequences.pkl',
    gene_sequences_path='data/gene_sequences_4000bp.pkl',
    batch_size=256,
    type='val',
    majority_fraction=0.005
)

test_loader = get_dataloader(
    parquet_path='data/tf_gene_expression_labeled_v2.parquet',
    tf_sequences_path='data/tf_sequences.pkl',
    gene_sequences_path='data/gene_sequences_4000bp.pkl',
    batch_size=256,
    type='test',
    majority_fraction=0.005
)

print('Train size:', len(train_loader.dataset))
print('Validation size :', len(validation_loader.dataset))
print('Test size :', len(test_loader.dataset))

Train size: 10845
Validation size : 2324
Test size : 2325


In [3]:
train_ds = train_loader.dataset
validation_ds = validation_loader.dataset
test_ds = test_loader.dataset

combined_df = pd.concat([train_ds.df, validation_ds.df, test_ds.df]).reset_index(drop=True)

# unique names from combined set
tf_names = combined_df['tf_name'].unique().tolist()
gene_names = combined_df['gene_name'].unique().tolist()

# create mappings
tf_to_id = {n: i for i, n in enumerate(tf_names)}
gene_to_id = {n: i for i, n in enumerate(gene_names)}

num_tfs = len(tf_to_id)
num_genes = len(gene_to_id)
# Use classes from training split
num_classes = len(train_ds.df['expression_label'].unique())

print('Unique TFs (combined):', num_tfs)
print('Unique Genes (combined):', num_genes)
print('Num classes:', num_classes)

Unique TFs (combined): 223
Unique Genes (combined): 4539
Num classes: 3


In [4]:
# load NT sequence embeddings
# tf_embed_cache = pickle.load(open("./embeds/tf_cls.pkl", "rb"))
# gene_embed_cache = pickle.load(open("./embeds/gn_cls.pkl", "rb"))

# Load enformer pca embeddings
tf_embed_cache = pickle.load(open("./embeds/tf_enformer_alternate_pca.pkl", "rb"))
gene_embed_cache = pickle.load(open("./embeds/gn_enformer_alternate_pca.pkl", "rb"))

# ensure everything is torch tensors
for k in tf_embed_cache:
    if not isinstance(tf_embed_cache[k], torch.Tensor):
        tf_embed_cache[k] = torch.tensor(tf_embed_cache[k], dtype=torch.float32)

for k in gene_embed_cache:
    if not isinstance(gene_embed_cache[k], torch.Tensor):
        gene_embed_cache[k] = torch.tensor(gene_embed_cache[k], dtype=torch.float32)

first_tf = next(iter(tf_embed_cache.values()))
first_gene = next(iter(gene_embed_cache.values()))
print("TF emb dim:", first_tf.shape)
print("Gene emb dim:", first_gene.shape)


TF emb dim: torch.Size([18])
Gene emb dim: torch.Size([26])


In [5]:
def one_hot(index, num_classes):
    v = torch.zeros(num_classes, dtype=torch.float32)
    v[index] = 1.0
    return v

In [37]:
def prepare_combined_one_hot(tf_name, gene_name):
    tf_id = tf_to_id[tf_name]
    gene_id = gene_to_id[gene_name]
    
    tf_one_hot = one_hot(tf_id, num_tfs)
    gene_one_hot = one_hot(gene_id, num_genes)
    
    tf_embed = tf_embed_cache[tf_name]
    gene_embed = gene_embed_cache[gene_name]

    interaction = torch.nn.functional.pad(tf_embed, (0, len(gene_embed)-len(tf_embed))) * gene_embed
    
    # combined_vector = torch.cat([tf_one_hot, gene_one_hot, tf_embed, gene_embed, interaction], dim=0)
    # combined_vector = torch.cat([tf_one_hot, gene_one_hot, tf_embed, gene_embed], dim=0)
    combined_vector = torch.cat([tf_embed, gene_embed, interaction], dim=0)

    return combined_vector

def prepare_combined_dataset(loader):
    X_list = []
    y_list = []
    
    for batch_x, batch_y in loader:
        for i in range(len(batch_x)):
            item = batch_x[i]
            combined_vector = prepare_combined_one_hot(item['tf_name'], item['gene_name'])
            X_list.append(combined_vector.numpy())
        
        y_list.extend(batch_y.numpy())
    
    X = np.array(X_list)
    y = np.array(y_list)
    return X, y

In [38]:
# Prepare training data
print("Preparing training data with combined one-hot and embeddings...")
X_train, y_train = prepare_combined_dataset(train_loader)
print(f"Train shape after combined: X={X_train.shape}, y={y_train.shape}")

# Prepare validation dataq
X_val, y_val = prepare_combined_dataset(validation_loader)
print(f"Validation shape after combined: X={X_val.shape}, y={y_val.shape}")

# Prepare test data
X_test, y_test = prepare_combined_dataset(test_loader)
print(f"Test shape after combined: X={X_test.shape}, y={y_test.shape}")

Preparing training data with combined one-hot and embeddings...
Train shape after combined: X=(10845, 70), y=(10845,)
Validation shape after combined: X=(2324, 70), y=(2324,)
Test shape after combined: X=(2325, 70), y=(2325,)


In [39]:
print(X_train[0])

[ 0.5348676  -3.1750827  -0.84219706 -2.0408177   1.6401079   1.1328499
 -2.522283   -2.0506234   1.1318072  -1.574157   -0.25275308  1.4961165
  1.8082484   0.27242145 -0.12476249  0.41143802  0.7936815   1.1141872
  5.8550024   0.8629087   4.367639    1.1178275  -2.3470953   1.3933285
 -3.3553195  -1.5448911   0.03683891  1.4410074   0.69558436  1.3582042
  0.44553939 -0.7737664  -0.19838746 -0.03068373  0.46957397  0.08014184
  0.12443868 -0.37248608 -0.07401615 -0.27711746 -0.26388505  0.6317827
  0.01573891 -0.20281078  3.131651   -2.7398067  -3.6784127  -2.2812822
 -3.8494895   1.5784322   8.463065    3.16799     0.04169454 -2.2683718
 -0.17581108  2.0320318   0.8056459  -0.21079056  0.02475131 -0.01262445
  0.37269217  0.08929302  0.         -0.         -0.         -0.
 -0.          0.          0.         -0.        ]


In [29]:
#!pip install xgboost

In [40]:
# XGBoost and metrics
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score
import json

In [86]:
# Cell 10 - Train XGBoost with one-hot encoded features
print("\nTraining XGBoost with combined encoding...")
# xgb_model = XGBClassifier(
#     n_estimators=200,          # More trees for high-dimensional data
#     max_depth=10,               # Moderate depth
#     learning_rate=0.1,         # Standard learning rate
#     subsample=0.8,             # Sample 80% of data
#     colsample_bytree=0.8,      # Sample 80% of features
#     reg_alpha=0.1,             # L1 regularization
#     reg_lambda=1.0,            # L2 regularization
#     min_child_weight=3,        # Minimum samples in leaf
#     gamma=0.01,                # Minimum loss reduction
#     random_state=10701,
#     n_jobs=-1,
#     tree_method='hist',        # Efficient for sparse data
#     eval_metric='mlogloss',
#     early_stopping_rounds=15,
#     verbosity=0
# )

# the following gets train f1 of 1, test f1 of 0.739
# xgb_model = XGBClassifier(n_estimators=150,
#                           max_depth=10,
#                           learning_rate=0.1,
#                           subsample=0.85,
#                           colsample_bytree=0.85, 
#                           reg_lambda=1.0,
#                           random_state=10701,
#                           n_jobs=-1,
#                           tree_method='hist',
#                           eval_metric='mlogloss',  
#                           early_stopping_rounds=15,
#                           verbosity=0)   

xgb_model = XGBClassifier(
    n_estimators=300,            # Further reduce (was 100)
    max_depth=15,                # Even shallower (was 6)
    learning_rate=0.1,         # Slower (was 0.05)
    subsample=0.8,              # More aggressive (was 0.6)
    colsample_bytree=0.5,       # More aggressive (was 0.6)
    colsample_bylevel=0.5,      # Keep aggressive
    colsample_bynode=0.5,       # Keep aggressive
    min_child_weight=10,        # Increase (was 10)
    gamma=1.0,                  # Increase (was 1.0)
    reg_alpha=3.0,              # Increase L1 (was 2.0)
    reg_lambda=3.0,             # Increase L2 (was 5.0)
    random_state=10701,
    n_jobs=-1,
    tree_method='hist',
    eval_metric='mlogloss',
    early_stopping_rounds=30,
    verbosity=0
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=10
)


Training XGBoost with combined encoding...
[0]	validation_0-mlogloss:1.07106
[10]	validation_0-mlogloss:0.90177
[20]	validation_0-mlogloss:0.82805
[30]	validation_0-mlogloss:0.78376
[40]	validation_0-mlogloss:0.75532
[50]	validation_0-mlogloss:0.73600
[60]	validation_0-mlogloss:0.72187
[70]	validation_0-mlogloss:0.71179
[80]	validation_0-mlogloss:0.70474
[90]	validation_0-mlogloss:0.70053
[100]	validation_0-mlogloss:0.69603
[110]	validation_0-mlogloss:0.69371
[120]	validation_0-mlogloss:0.69167
[130]	validation_0-mlogloss:0.68894
[140]	validation_0-mlogloss:0.68764
[150]	validation_0-mlogloss:0.68553
[160]	validation_0-mlogloss:0.68467
[170]	validation_0-mlogloss:0.68378
[180]	validation_0-mlogloss:0.68280
[190]	validation_0-mlogloss:0.68175
[200]	validation_0-mlogloss:0.68093
[210]	validation_0-mlogloss:0.68026
[220]	validation_0-mlogloss:0.67958
[230]	validation_0-mlogloss:0.67931
[240]	validation_0-mlogloss:0.67890
[250]	validation_0-mlogloss:0.67859
[260]	validation_0-mlogloss:0.6

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,0.5
,colsample_bynode,0.5
,colsample_bytree,0.5
,device,None
,early_stopping_rounds,30
,enable_categorical,False
,eval_metric,'mlogloss'


In [87]:
# Evaluate on all sets
for name, X, y in [('Train', X_train, y_train), 
                    ('Val', X_val, y_val), 
                    ('Test', X_test, y_test)]:
    y_pred = xgb_model.predict(X)
    acc = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    print(f"\n=== {name} Set ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"Macro F1: {f1:.4f}")
    print("Classification Report:")
    print(classification_report(y, y_pred, digits=4))


=== Train Set ===
Accuracy: 0.8834
Macro F1: 0.8845
Classification Report:
              precision    recall  f1-score   support

           0     0.8785    0.8704    0.8745      3573
           1     0.9122    0.9331    0.9226      3409
           2     0.8614    0.8514    0.8564      3863

    accuracy                         0.8834     10845
   macro avg     0.8841    0.8850    0.8845     10845
weighted avg     0.8830    0.8834    0.8831     10845


=== Val Set ===
Accuracy: 0.7044
Macro F1: 0.7060
Classification Report:
              precision    recall  f1-score   support

           0     0.6658    0.6945    0.6799       766
           1     0.8028    0.8589    0.8299       730
           2     0.6425    0.5773    0.6081       828

    accuracy                         0.7044      2324
   macro avg     0.7037    0.7102    0.7060      2324
weighted avg     0.7005    0.7044    0.7014      2324


=== Test Set ===
Accuracy: 0.7101
Macro F1: 0.7115
Classification Report:
             

# Grid search

In [11]:
# Manual Grid Search with tqdm
from sklearn.model_selection import cross_val_score
from itertools import product
import numpy as np

param_grid = {
    'n_estimators': [150, 250],
    'max_depth': [8, 12],
    'learning_rate': [0.05, 0.15],
    'subsample': [0.7, 0.85],
    'colsample_bytree': [0.7, 0.85],
    'reg_lambda': [1.0, 2.0]
    # Removed min_child_weight and reg_alpha to reduce combinations
}

# Generate all combinations
keys = list(param_grid.keys())
values = list(param_grid.values())
param_combinations = [dict(zip(keys, v)) for v in product(*values)]

print(f"Total combinations to test: {len(param_combinations)}")

# Manual grid search with progress bar
best_score = -np.inf
best_params = None
results = []

for params in tqdm(param_combinations, desc="Grid Search"):
    model = XGBClassifier(
        **params,
        random_state=10701,
        n_jobs=-1,
        tree_method='hist',
        eval_metric='mlogloss',
        verbosity=0
    )
    
    # Cross-validation
    scores = cross_val_score(
        model, X_train, y_train,
        cv=3,
        scoring='f1_macro',
        n_jobs=1  # Important: don't parallelize CV
    )
    
    mean_score = scores.mean()
    std_score = scores.std()
    
    results.append({
        'params': params,
        'mean_f1': mean_score,
        'std_f1': std_score
    })
    
    if mean_score > best_score:
        best_score = mean_score
        best_params = params
        print(f"  New best! F1={mean_score:.4f} (±{std_score:.4f})")

print("\n✓ Grid Search Complete!")
print(f"Best CV F1: {best_score:.4f}")
print(f"Best parameters: {best_params}")

# Train final model with best params
print("\nTraining final model with best parameters...")
xgb_model = XGBClassifier(
    **best_params,
    random_state=10701,
    n_jobs=-1,
    tree_method='hist',
    eval_metric='mlogloss',
    verbosity=0
)

xgb_model.fit(X_train, y_train)

# Evaluate
for name, X, y in [('Train', X_train, y_train), 
                    ('Val', X_val, y_val), 
                    ('Test', X_test, y_test)]:
    y_pred = xgb_model.predict(X)
    acc = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    print(f"\n=== {name} Set ===")
    print(f"Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

# Save results
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('mean_f1', ascending=False)
results_df.to_csv('results/grid_search_results.csv', index=False)
print("\nTop 10 parameter combinations saved to results/grid_search_results.csv")
print(results_df.head(10))

Total combinations to test: 64


Grid Search:   2%|▏         | 1/64 [00:16<17:23, 16.56s/it]

  New best! F1=0.5999 (±0.0139)


Grid Search:   5%|▍         | 3/64 [00:57<19:47, 19.47s/it]

  New best! F1=0.6020 (±0.0134)


...

  New best! F1=0.6874 (±0.0066)


Grid Search:  98%|█████████▊| 63/64 [22:14<00:24, 24.87s/it]

  New best! F1=0.6882 (±0.0061)


Grid Search: 100%|██████████| 64/64 [22:39<00:00, 21.24s/it]



✓ Grid Search Complete!
Best CV F1: 0.6882
Best parameters: {'n_estimators': 250, 'max_depth': 12, 'learning_rate': 0.15, 'subsample': 0.85, 'colsample_bytree': 0.85, 'reg_lambda': 1.0}

Training final model with best parameters...

=== Train Set ===
Accuracy: 0.7517, Macro F1: 0.7516

=== Val Set ===
Accuracy: 0.6936, Macro F1: 0.6904

=== Test Set ===
Accuracy: 0.7191, Macro F1: 0.7161

Top 10 parameter combinations saved to results/grid_search_results.csv
                                               params   mean_f1    std_f1
62  {'n_estimators': 250, 'max_depth': 12, 'learni...  0.688155  0.006108
60  {'n_estimators': 250, 'max_depth': 12, 'learni...  0.687379  0.006591
58  {'n_estimators': 250, 'max_depth': 12, 'learni...  0.686594  0.006224
56  {'n_estimators': 250, 'max_depth': 12, 'learni...  0.685724  0.005347
63  {'n_estimators': 250, 'max_depth': 12, 'learni...  0.681179  0.007222
61  {'n_estimators': 250, 'max_depth': 12, 'learni...  0.680978  0.007099
57  {'n_estimators